## Reachy expressif correction 

In [4]:
import random 
import pygame 
import time
import asyncio

from reachy_sdk import ReachySDK
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory import goto_async
from reachy_sdk.trajectory.interpolation import InterpolationMode

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
reachy = ReachySDK('localhost')
pygame.init()

pygame.mixer.init()

Commande pygame pour gérer le son : 

In [ ]:
pygame.mixer.music.load #permet de charger un fichier de fond sonore.
pygame.mixer.music.play #permet de jouer/lire le fond sonore.
pygame.mixer.music.rewind #permet de reprendre au début le fond sonore.
pygame.mixer.music.stop #permet d’arrêter la lecture du fond sonore.
pygame.mixer.music.pause #permet de faire une pause dans la lecture.
pygame.mixer.music.set_volume #permet de régler le volume du fond sonore.
pygame.mixer.music.get_volume #permet de connaître le volume courant du fond sonore.

In [56]:
recorded_joints = [
    reachy.r_arm.r_shoulder_pitch,
    reachy.r_arm.r_shoulder_roll,
    reachy.r_arm.r_arm_yaw,
    reachy.r_arm.r_elbow_pitch,
    reachy.r_arm.r_forearm_yaw,
    reachy.r_arm.r_wrist_pitch,
    reachy.r_arm.r_wrist_roll,
    reachy.l_arm.l_shoulder_pitch,
    reachy.l_arm.l_shoulder_roll,
    reachy.l_arm.l_arm_yaw,
    reachy.l_arm.l_elbow_pitch,
    reachy.l_arm.l_forearm_yaw,
    reachy.l_arm.l_wrist_pitch,
    reachy.l_arm.l_wrist_roll,
    reachy.head.joints.neck_disk_top,
    reachy.head.joints.neck_disk_middle,
    reachy.head.joints.neck_disk_bottom,
]

sampling_frequency = 100  #en hertz
record_duration = 15  #en seconde

### La classe emotion : 

In [90]:
class Emotion : 
    def __init__(self, nom, traject, son):
        self.__nom = nom 
        self.__traject = traject
        self.__son = son
            
    def trajectoriesPlayer(self):

        #on rend rigide toutes jointures utiliser pour effectuer la trajectoire
        reachy.turn_on('reachy')

        #on créer un dictionnaire associant chaque jointure à sa première position 
        first_point = dict(zip(recorded_joints, self.__traject[0]))

        #le robot se positionne sur les premieres positions de chaque jointures 
        goto(first_point, duration=3.0)

        for joints_positions in self.__traject:
            for joint, pos in zip(recorded_joints, joints_positions):
                joint.goal_position = pos

            time.sleep(1 / sampling_frequency)

        reachy.turn_off_smoothly('reachy')
        
    def playEmotion(self):
        self.trajectoriesPlayer()
        pygame.mixer.music.play(1,0.0)
        
    
    def printName(self):
        print("L'émotion est =", self.__nom)

### Enregistrer les mouvements pour les émotions : 

In [91]:
listEmotion = []
reachy.turn_off('reachy')
trajectories = [] #on créer une nouvelle liste trajectoire
for e in range(2) : 
    print(f'ATTENTION : lancement du recorde de la {e+1}ème émotion')
    emotion = input("Entrer l'émotion que vous souhaiter enregistrer :")
    trajectories.clear()
    start = time.time() #seconde passées depuis epoch
    while (time.time() - start) < record_duration:
        #on optient les positions actuelles de toutes les jointures 
        current_point = [joint.present_position for joint in recorded_joints]
        #on ajoute les positions à la liste trajectoire 
        trajectories.append(current_point)
        time.sleep(1 / sampling_frequency)
    son = pygame.mixer.music.load('/home/reachy/Reachy_Nautilus/Sons/sonar.ogg')
    emotion = Emotion(emotion, trajectories, son)
    print(trajectories)
    listEmotion.append(emotion)
    

ATTENTION : lancement du recorde de la 1ème émotion
Entrer l'émotion que vous souhaiter enregistrer :peur
[[-8.06, 0.02, 21.67, -58.95, -10.12, -28.26, 12.76, -6.66, -1.96, -20.97, -57.45, -4.25, -39.78, -33.28, -45.07, -108.15, -44.18], [-8.06, 0.02, 21.67, -58.95, -10.12, -28.26, 12.76, -6.66, -1.96, -20.97, -57.45, -4.25, -39.78, -33.28, -45.07, -108.15, -44.18], [-8.06, 0.11, 21.67, -58.95, -10.12, -28.26, 12.76, -6.66, -2.05, -20.97, -57.45, -4.25, -39.78, -33.28, -45.07, -108.15, -44.18], [-8.06, 0.11, 21.67, -58.95, -10.12, -28.26, 12.76, -6.66, -2.05, -20.97, -57.45, -4.25, -39.78, -33.28, -45.07, -108.15, -44.18], [-8.06, 0.11, 21.67, -58.95, -10.12, -28.26, 12.76, -6.66, -1.96, -20.97, -57.45, -4.25, -39.78, -33.28, -45.07, -108.15, -44.18], [-8.06, 0.11, 21.67, -58.95, -10.12, -28.26, 12.76, -6.66, -2.05, -20.97, -57.45, -4.25, -39.78, -33.28, -45.07, -108.15, -44.18], [-8.06, 0.11, 21.67, -58.95, -10.12, -28.26, 12.76, -6.66, -2.05, -20.97, -57.45, -4.25, -39.78, -33.28, -4

Entrer l'émotion que vous souhaiter enregistrer :tristesse
[[-10.0, -5.78, 25.01, -58.95, 0.73, -22.02, 15.1, -18.35, -0.02, -9.1, -44.7, -17.45, -36.26, -28.3, -45.07, -108.15, -44.18], [-10.0, -5.78, 25.01, -58.95, 0.73, -22.02, 15.1, -18.35, -0.02, -9.1, -44.7, -17.45, -36.26, -28.3, -45.07, -108.15, -44.18], [-10.0, -5.78, 25.01, -58.95, 0.73, -22.02, 15.1, -18.35, -0.02, -9.1, -44.7, -17.45, -36.26, -28.3, -45.07, -108.15, -44.18], [-10.0, -5.78, 25.01, -58.95, 0.73, -22.02, 15.1, -18.35, -0.02, -9.1, -44.7, -17.45, -36.26, -28.3, -45.07, -108.15, -44.18], [-10.0, -5.78, 25.01, -58.95, 0.73, -22.02, 15.1, -18.35, -0.02, -9.1, -44.7, -17.45, -36.26, -28.3, -45.07, -108.15, -44.18], [-10.0, -5.78, 25.01, -58.95, 0.73, -22.02, 15.1, -18.35, -0.02, -9.1, -44.7, -17.45, -36.26, -28.3, -45.07, -108.15, -44.18], [-10.0, -5.78, 25.01, -58.95, 0.73, -22.02, 15.1, -18.35, -0.02, -9.19, -44.7, -17.45, -36.26, -28.3, -45.07, -108.15, -44.18], [-10.0, -5.78, 25.01, -58.95, 0.73, -22.02, 15.1, 

In [67]:
trajectories

[[-12.2,
  -4.2,
  19.82,
  -58.95,
  -6.89,
  -14.46,
  17.16,
  -12.64,
  -3.1,
  -1.27,
  -52.18,
  -23.61,
  -31.08,
  -18.04,
  -151.67,
  -55.74,
  -100.36],
 [-12.2,
  -4.2,
  19.82,
  -58.95,
  -6.89,
  -14.46,
  17.16,
  -12.64,
  -3.01,
  -1.27,
  -52.18,
  -23.61,
  -31.08,
  -18.04,
  -151.67,
  -55.74,
  -100.36],
 [-12.2,
  -4.2,
  19.82,
  -58.95,
  -6.89,
  -14.46,
  17.16,
  -12.64,
  -3.01,
  -1.27,
  -52.18,
  -23.61,
  -31.08,
  -18.04,
  -151.67,
  -55.74,
  -100.36],
 [-12.2,
  -4.2,
  19.91,
  -58.95,
  -6.89,
  -14.37,
  17.16,
  -12.64,
  -3.01,
  -1.27,
  -52.26,
  -23.61,
  -31.08,
  -18.04,
  -151.67,
  -55.74,
  -100.36],
 [-12.2,
  -4.2,
  19.91,
  -58.95,
  -6.89,
  -14.37,
  17.16,
  -12.64,
  -3.01,
  -1.27,
  -52.26,
  -23.61,
  -31.08,
  -18.04,
  -151.67,
  -55.74,
  -100.36],
 [-12.2,
  -4.2,
  19.82,
  -58.95,
  -6.89,
  -14.37,
  17.16,
  -12.64,
  -3.1,
  -1.27,
  -52.26,
  -23.61,
  -31.08,
  -18.04,
  -151.67,
  -55.74,
  -100.36],
 [-12.2,
  -

In [93]:
reachy.turn_on('reachy')
listEmotion[1].printName()
listEmotion[1].playEmotion()

L'émotion est = tristesse


* Jointures à enregistrer (les 2 bras et la tête) : 

In [9]:
reachy.turn_off('reachy')

Lancement de l'enregistrement, aprête toi a bouger la partie du reachy comme tu le souhaite 

In [10]:
trajectories = [] #on créer une nouvelle liste trajectoire

start = time.time() #seconde passées depuis epoch
while (time.time() - start) < record_duration:
    #on optient les positions actuelles de toutes les jointures 
    current_point = [joint.present_position for joint in recorded_joints]
    #on ajoute les positions à la liste trajectoire 
    trajectories.append(current_point)

    time.sleep(1 / sampling_frequency)

Si tu veux vérifier ta trajectoire, exécute cette cellule qui rejouera ta trajectoire : 

In [11]:
#on rend rigide toutes jointures utiliser pour effectuer la trajectoire
reachy.turn_on('reachy')

#on créer un dictionnaire associant chaque jointure à sa première position 
first_point = dict(zip(recorded_joints, trajectories[0]))

#le robot se positionne sur les premieres positions de chaque jointures 
goto(first_point, duration=3.0)

for joints_positions in trajectories:
    for joint, pos in zip(recorded_joints, joints_positions):
        joint.goal_position = pos

    time.sleep(1 / sampling_frequency)

reachy.turn_off_smoothly('reachy')

L'enregistrement du mouvement s'apelle *trajectories*, c'est cela que tu dois rentrer comme 2ième argument lorsque tu créer une nouvelle émotion avec la classe Emotion

## Création des instances de la classe Emotion (les différentes émotions) : 

In [17]:
emotion = input("Entrer l'émotion que vous souhaiter enregistrer :")

Entrer l'émotion que vous souhaiter enregistrer :pouet


Création d'une nouvelle instance de l'objet émotion : 

In [32]:
emotion = input("Entrer l'émotion que vous souhaiter enregistrer :")
son = pygame.mixer.music.load('/home/reachy/Reachy_Nautilus/Sons/sonar.ogg')
emotion = Emotion(emotion, trajectories, son)
listEmotion = []
listEmotion.append(emotion)


Entrer l'émotion que vous souhaiter enregistrer :peur


In [33]:
listEmotion

Jouer l'émotion crée :

In [19]:
emotion.playEmotion()

Permet de générer une séquence d'émotions choisi au hazard : 

In [25]:
emotion = ['peur', 'joie', 'tristesse', 'colere', 'surprise']
sequence = []
for i in range(5):
    sequence.append(random.choice(emotion))

print("Séquence d'émotion choisie au hasard :", sequence)

Séquence d'émotion choisie au hasard : ['colere', 'peur', 'tristesse', 'peur', 'surprise']


In [28]:
if sequence[1] == 'peur':
    peur.playEmotion()

In [27]:
for e in range(len(sequence)):
    sequence[e].playEmotion()
    print(f'Emotion jouée actuellement: {sequence.printName()}')
    

AttributeError: 'str' object has no attribute 'playEmotion'

In [ ]:
reponse = input("Entrez dans l'ordre les émotions de la séquence vu auparavant : ")

In [ ]:
if reponse == sequence :
    'gagné!'